<a href="https://colab.research.google.com/github/MarcinK420/MEDICA/blob/main/ZarzadzanieZadanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# W komórce Colab
!pip install Flask pyngrok Werkzeug==2.3.7  # Instalujemy Flask i ngrok (do wystawienia aplikacji na zewnątrz) oraz konkretną wersję Werkzeug dla kompatybilności

INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
# W komórce Colab
import os

# Tworzymy katalog na szablony HTML
if not os.path.exists('templates'):
    os.makedirs('templates')

# Tworzymy plik bazy danych (jeśli nie istnieje)
import sqlite3
conn = sqlite3.connect('patients.db')
conn.close()

In [ ]:
# W kolejnej komórce Colab - zapiszemy to jako plik app.py
%%writefile app.py
import os
import sqlite3
import hashlib # Do hashowania haseł
from flask import Flask, render_template, request, redirect, url_for, session, flash

app = Flask(__name__)
# Klucz sekretny jest potrzebny do obsługi sesji (logowania)
app.secret_key = os.urandom(24) # Generuje losowy klucz przy każdym uruchomieniu

DATABASE = 'patients.db'

# --- Funkcje pomocnicze do bazy danych ---

def get_db():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row # Zwraca wiersze jako obiekty podobne do słowników
    return conn

def init_db():
    """Inicjalizuje bazę danych, tworząc tabelę jeśli nie istnieje."""
    db = get_db()
    cursor = db.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS patients (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL,
            pesel TEXT UNIQUE,  -- Dodajmy PESEL jako przykład
            date_of_birth TEXT
        )
    ''')
    db.commit()
    cursor.close()
    db.close()
    print("Database initialized.")

def hash_password(password):
    """Haszuje hasło za pomocą SHA256."""
    return hashlib.sha256(password.encode('utf-8')).hexdigest()

def verify_password(stored_password_hash, provided_password):
    """Weryfikuje podane hasło z hashem zapisanym w bazie."""
    return stored_password_hash == hashlib.sha256(provided_password.encode('utf-8')).hexdigest()

# --- Trasy (Routes) aplikacji ---

@app.route('/')
def index():
    """Strona główna - przekierowuje do logowania lub karty pacjenta."""
    if 'patient_id' in session:
        return redirect(url_for('patient_card'))
    return redirect(url_for('login'))

@app.route('/register', methods=['GET', 'POST'])
def register():
    """Strona rejestracji nowego pacjenta."""
    if request.method == 'POST':
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        email = request.form['email']
        password = request.form['password']
        pesel = request.form.get('pesel') # Użyj get dla pól opcjonalnych
        date_of_birth = request.form.get('date_of_birth')

        if not all([first_name, last_name, email, password]):
             flash('Wszystkie pola (oprócz PESEL i daty urodzenia) są wymagane!', 'error')
             return redirect(url_for('register'))

        password_hash = hash_password(password)

        conn = get_db()
        cursor = conn.cursor()
        try:
            cursor.execute('''
                INSERT INTO patients (first_name, last_name, email, password_hash, pesel, date_of_birth)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (first_name, last_name, email, password_hash, pesel, date_of_birth))
            conn.commit()
            flash('Rejestracja zakończona sukcesem! Możesz się teraz zalogować.', 'success')
            return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            flash('Użytkownik o podanym adresie e-mail lub PESEL już istnieje.', 'error')
            return redirect(url_for('register'))
        finally:
            cursor.close()
            conn.close()

    # Jeśli metoda GET, po prostu wyświetl formularz
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    """Strona logowania pacjenta."""
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        conn = get_db()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM patients WHERE email = ?', (email,))
        patient = cursor.fetchone()
        cursor.close()
        conn.close()

        if patient and verify_password(patient['password_hash'], password):
            session['patient_id'] = patient['id']
            session['patient_email'] = patient['email']
            flash('Zalogowano pomyślnie!', 'success')
            return redirect(url_for('patient_card'))
        else:
            flash('Nieprawidłowy e-mail lub hasło.', 'error')
            return redirect(url_for('login'))

    # Jeśli metoda GET, wyświetl formularz logowania
    return render_template('login.html')

@app.route('/patient_card')
def patient_card():
    """Wyświetla kartę pacjenta dla zalogowanego użytkownika."""
    if 'patient_id' not in session:
        flash('Musisz być zalogowany, aby zobaczyć tę stronę.', 'error')
        return redirect(url_for('login'))

    patient_id = session['patient_id']
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM patients WHERE id = ?', (patient_id,))
    patient_data = cursor.fetchone()
    cursor.close()
    conn.close()

    if not patient_data:
        # Sytuacja awaryjna, użytkownik w sesji, ale nie ma go w bazie?
        session.pop('patient_id', None)
        session.pop('patient_email', None)
        flash('Nie znaleziono danych pacjenta. Zaloguj się ponownie.', 'error')
        return redirect(url_for('login'))

    return render_template('patient_card.html', patient=patient_data)

@app.route('/logout')
def logout():
    """Wylogowuje użytkownika."""
    session.pop('patient_id', None)
    session.pop('patient_email', None)
    flash('Wylogowano pomyślnie.', 'info')
    return redirect(url_for('login'))

# --- Główna część skryptu ---
if __name__ == '__main__':
    init_db() # Upewnij się, że tabela istnieje przy starcie

    # ---- Dodaj konfigurację ngrok ----
    from pyngrok import ngrok, conf

    # Opcjonalnie: Ustaw region ngrok bliżej siebie (np. Europa)
    # Dostępne regiony: us, eu, ap, au, sa, jp, in
    conf.get_default().region = 'eu'

    # Ustawienie portu (musi być taki sam jak w app.run i ngrok.connect)
    port = 5000

    print("Starting Flask app with ngrok...")
    # Otwórz tunel ngrok na porcie, na którym będzie działać Flask
    public_url = ngrok.connect(port)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}/\"")

    # Uruchom aplikację Flask na tym samym porcie
    # Użyj host='0.0.0.0', aby była dostępna z zewnątrz (dla ngrok)
    app.run(host='0.0.0.0', port=port)
    # ------------------------------------

Overwriting app.py


In [ ]:
%%writefile templates/register.html
<!DOCTYPE html>
<html lang="pl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Rejestracja Pacjenta</title>
    <style>
        body { font-family: sans-serif; margin: 20px; }
        .form-group { margin-bottom: 15px; }
        label { display: block; margin-bottom: 5px; }
        input[type="text"], input[type="email"], input[type="password"], input[type="date"] {
            width: 300px; padding: 8px; border: 1px solid #ccc; border-radius: 4px;
        }
        button { padding: 10px 15px; background-color: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer; }
        button:hover { background-color: #0056b3; }
        .flash { padding: 10px; margin-bottom: 15px; border-radius: 4px; }
        .flash.success { background-color: #d4edda; color: #155724; border: 1px solid #c3e6cb; }
        .flash.error { background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }
        .flash.info { background-color: #d1ecf1; color: #0c5460; border: 1px solid #bee5eb; }
        a { color: #007bff; text-decoration: none; }
        a:hover { text-decoration: underline; }
    </style>
</head>
<body>
    <h1>Rejestracja Nowego Pacjenta</h1>

    {% with messages = get_flashed_messages(with_categories=true) %}
      {% if messages %}
        {% for category, message in messages %}
          <div class="flash {{ category }}">{{ message }}</div>
        {% endfor %}
      {% endif %}
    {% endwith %}

    <form method="POST" action="{{ url_for('register') }}">
        <div class="form-group">
            <label for="first_name">Imię:</label>
            <input type="text" id="first_name" name="first_name" required>
        </div>
        <div class="form-group">
            <label for="last_name">Nazwisko:</label>
            <input type="text" id="last_name" name="last_name" required>
        </div>
        <div class="form-group">
            <label for="email">Adres E-mail:</label>
            <input type="email" id="email" name="email" required>
        </div>
         <div class="form-group">
            <label for="pesel">PESEL (opcjonalnie):</label>
            <input type="text" id="pesel" name="pesel" pattern="\d{11}" title="PESEL musi składać się z 11 cyfr">
        </div>
         <div class="form-group">
            <label for="date_of_birth">Data urodzenia (opcjonalnie):</label>
            <input type="date" id="date_of_birth" name="date_of_birth">
        </div>
        <div class="form-group">
            <label for="password">Hasło:</label>
            <input type="password" id="password" name="password" required>
        </div>
        <button type="submit">Zarejestruj</button>
    </form>
    <p>Masz już konto? <a href="{{ url_for('login') }}">Zaloguj się tutaj</a>.</p>
</body>
</html>

Writing templates/register.html


In [ ]:
%%writefile templates/login.html
<!DOCTYPE html>
<html lang="pl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Logowanie Pacjenta</title>
    <style>
        body { font-family: sans-serif; margin: 20px; }
        .form-group { margin-bottom: 15px; }
        label { display: block; margin-bottom: 5px; }
        input[type="email"], input[type="password"] {
            width: 300px; padding: 8px; border: 1px solid #ccc; border-radius: 4px;
        }
        button { padding: 10px 15px; background-color: #28a745; color: white; border: none; border-radius: 4px; cursor: pointer; }
        button:hover { background-color: #218838; }
        .flash { padding: 10px; margin-bottom: 15px; border-radius: 4px; }
        .flash.success { background-color: #d4edda; color: #155724; border: 1px solid #c3e6cb; }
        .flash.error { background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }
        .flash.info { background-color: #d1ecf1; color: #0c5460; border: 1px solid #bee5eb; }
        a { color: #007bff; text-decoration: none; }
        a:hover { text-decoration: underline; }
    </style>
</head>
<body>
    <h1>Logowanie do Systemu</h1>

    {% with messages = get_flashed_messages(with_categories=true) %}
      {% if messages %}
        {% for category, message in messages %}
          <div class="flash {{ category }}">{{ message }}</div>
        {% endfor %}
      {% endif %}
    {% endwith %}

    <form method="POST" action="{{ url_for('login') }}">
        <div class="form-group">
            <label for="email">Adres E-mail:</label>
            <input type="email" id="email" name="email" required>
        </div>
        <div class="form-group">
            <label for="password">Hasło:</label>
            <input type="password" id="password" name="password" required>
        </div>
        <button type="submit">Zaloguj</button>
    </form>
     <p>Nie masz konta? <a href="{{ url_for('register') }}">Zarejestruj się tutaj</a>.</p>
</body>
</html>

Writing templates/login.html


In [ ]:
%%writefile templates/patient_card.html
<!DOCTYPE html>
<html lang="pl">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Karta Pacjenta</title>
     <style>
        body { font-family: sans-serif; margin: 20px; }
        .card { border: 1px solid #ccc; border-radius: 5px; padding: 20px; max-width: 500px; background-color: #f9f9f9; }
        h1, h2 { color: #333; }
        p { margin-bottom: 10px; }
        strong { color: #555; }
        a { color: #dc3545; text-decoration: none; display: inline-block; margin-top: 15px; padding: 8px 12px; background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 4px;}
        a:hover { background-color: #f5c6cb; text-decoration: none;}
        .flash { padding: 10px; margin-bottom: 15px; border-radius: 4px; }
        .flash.success { background-color: #d4edda; color: #155724; border: 1px solid #c3e6cb; }
        .flash.error { background-color: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }
        .flash.info { background-color: #d1ecf1; color: #0c5460; border: 1px solid #bee5eb; }
    </style>
</head>
<body>
    <h1>Karta Pacjenta</h1>

    {% with messages = get_flashed_messages(with_categories=true) %}
      {% if messages %}
        {% for category, message in messages %}
          <div class="flash {{ category }}">{{ message }}</div>
        {% endfor %}
      {% endif %}
    {% endwith %}

    {% if patient %}
    <div class="card">
        <h2>Dane Pacjenta</h2>
        <p><strong>ID:</strong> {{ patient.id }}</p>
        <p><strong>Imię:</strong> {{ patient.first_name }}</p>
        <p><strong>Nazwisko:</strong> {{ patient.last_name }}</p>
        <p><strong>E-mail:</strong> {{ patient.email }}</p>
        <p><strong>PESEL:</strong> {{ patient.pesel if patient.pesel else 'Nie podano' }}</p>
        <p><strong>Data urodzenia:</strong> {{ patient.date_of_birth if patient.date_of_birth else 'Nie podano' }}</p>
        </div>
    {% else %}
    <p>Brak danych pacjenta do wyświetlenia.</p>
    {% endif %}

    <a href="{{ url_for('logout') }}">Wyloguj</a>
</body>
</html>

Writing templates/patient_card.html


In [ ]:
!ngrok config add-authtoken 2vl7i7VauDdaXwnGOohqOLyGlFO_6tsdW2gDTaxHvEXFU3dmY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!python app.py


Database initialized.
Starting Flask app with ngrok...
 * ngrok tunnel "NgrokTunnel: "https://0202-34-83-246-71.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
t=2025-04-15T09:07:05+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"
127.0.0.1 - - [15/Apr/2025 09:07:18] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [15/Apr/2025 09:07:18] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 09:07:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Apr/2025 09:08:06] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [15/Apr/2025 09:08:06] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 09:08:08] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2025 09:09:22] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [15/